In [6]:
import warnings
import logging
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Silenciar warnings y configurar logs
warnings.filterwarnings("ignore")
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)
datasets_logger = logging.getLogger("datasets")
datasets_logger.setLevel(logging.ERROR)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"

# Confguración para usar la GPU disponible (RTX 4070 SUPER)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device} ({torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'})")

df = pd.read_csv("../data/mails_dataset.csv")

# Combinar asunto y cuerpo con separador especial
df['text_combined'] = df['subject'].fillna('') + ' </s> ' + df['text'].fillna('')
df = df[['text_combined', 'priority']].dropna()

# Codificar categorías
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["priority"])
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}

# Crear Dataset HuggingFace
dataset = Dataset.from_pandas(df.rename(columns={"text_combined": "text", "label": "label"}))

# Tokenizar texto
model_name = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.train_test_split(test_size=0.2)

num_labels = len(label2id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Entrenamiento
training_args = TrainingArguments(
    output_dir="./priority_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs_priority",
    logging_steps=10,
    fp16=True,
    report_to="none"
)

# Evaluación
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

# Guardar modelo y tokenizer
trainer.save_model("./priority_model")
tokenizer.save_pretrained("./priority_model")


Usando dispositivo: cuda (NVIDIA GeForce RTX 4070 SUPER)


Map:   0%|          | 0/262 [00:00<?, ? examples/s]

{'loss': 1.0447, 'grad_norm': 4.813197135925293, 'learning_rate': 1.761904761904762e-05, 'epoch': 0.7142857142857143}
{'eval_loss': 0.8730468153953552, 'eval_accuracy': 0.6981132075471698, 'eval_f1': 0.6908805031446541, 'eval_runtime': 0.073, 'eval_samples_per_second': 726.021, 'eval_steps_per_second': 54.794, 'epoch': 1.0}
{'loss': 0.9076, 'grad_norm': 5.208207130432129, 'learning_rate': 1.5476190476190476e-05, 'epoch': 1.4285714285714286}
{'eval_loss': 0.6862769722938538, 'eval_accuracy': 0.7735849056603774, 'eval_f1': 0.7699889219082452, 'eval_runtime': 0.063, 'eval_samples_per_second': 841.277, 'eval_steps_per_second': 63.493, 'epoch': 2.0}
{'loss': 0.6144, 'grad_norm': 4.731895923614502, 'learning_rate': 1.3095238095238096e-05, 'epoch': 2.142857142857143}
{'loss': 0.5519, 'grad_norm': 3.9036455154418945, 'learning_rate': 1.0714285714285714e-05, 'epoch': 2.857142857142857}
{'eval_loss': 0.5854054093360901, 'eval_accuracy': 0.7547169811320755, 'eval_f1': 0.7487870619946091, 'eval_ru

('./priority_model\\tokenizer_config.json',
 './priority_model\\special_tokens_map.json',
 './priority_model\\tokenizer.json')

In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Obtener predicciones en el conjunto de validación
predictions = trainer.predict(dataset["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Mostrar el classification report con nombres reales
print("Reporte de clasificación por clase:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


Reporte de clasificación por clase:
              precision    recall  f1-score   support

        alta       0.86      1.00      0.92        12
        baja       0.78      0.74      0.76        19
       media       0.81      0.77      0.79        22

    accuracy                           0.81        53
   macro avg       0.81      0.84      0.82        53
weighted avg       0.81      0.81      0.81        53



In [ ]:
from transformers import pipeline
import pandas as pd

# Cargar modelo y tokenizer desde HuggingFace
clf_priority = pipeline("text-classification", model="aaronmena02/priority-model-mailclassifier", tokenizer="aaronmena02/priority-model-mailclassifier")

# Ejemplos para pruebas
ejemplos_priority = [
    ("Pedido no entregado", "Mi pedido no ha llegado y ya pasaron 10 días", "alta"),
    ("Repartidor descortés", "El repartidor fue descortés", "media"),
    ("Consulta sobre stock", "¿Pueden confirmarme si tienen stock del producto?", "media"),
    ("Agradecimiento", "Gracias por la atención, todo perfecto", "baja"),
    ("Caja vacía", "Recibí una caja vacía sin el producto dentro", "alta"),
    ("Modificar dirección", "Quisiera modificar la dirección de entrega", "media"),
    ("Alianza comercial", "Nos gustaría evaluar una posible alianza comercial", "baja"),
    ("Web confusa", "El servicio fue correcto, pero la web es un poco confusa", "media"),
    ("Alta urgente", "Solicito el alta inmediata del servicio para un cliente", "alta"),
    ("Correo no recibido", "No me llegó el correo de activación, pero puedo esperar", "baja"),
    ("Consulta sobre factura", "¿Podrían enviarme la factura del último pedido, por favor?", "media"),
    ("Problema técnico leve", "He tenido que reiniciar la app dos veces esta semana", "media"),
    ("Petición de reembolso", "Quiero tramitar el reembolso por el producto que no funcionó", "alta"),
    ("Comentario general", "Todo funciona bien, aunque echo de menos algunas funciones", "baja"),
    ("Cambio de titular", "Me gustaría cambiar el nombre de titular en la cuenta", "media"),
]

# Mostrar resultados
print(f"{'Asunto + Cuerpo':<70} | {'Esperado':<10} | {'Predicción':<10}")
print("-" * 115)
aciertos = 0

for asunto, cuerpo, esperado in ejemplos_priority:
    texto = f"{asunto.strip()}. {cuerpo.strip()}"
    pred = clf_priority(texto)[0]
    print(f"{texto[:67]:<70} | {esperado:<10} | {pred['label']:<10}")
    if pred['label'] == esperado:
        aciertos += 1

porcentaje_acierto = (aciertos / len(ejemplos_priority)) * 100
print("-" * 115)
print(f"Aciertos: {aciertos} / {len(ejemplos_priority)}  →  Precisión: {porcentaje_acierto:.2f}%")


Asunto + Cuerpo                                                        | Esperado   | Predicción
-------------------------------------------------------------------------------------------------------------------
Pedido no entregado. Mi pedido no ha llegado y ya pasaron 10 días      | alta       | alta      
Repartidor descortés. El repartidor fue descortés                      | media      | baja      
Consulta sobre stock. ¿Pueden confirmarme si tienen stock del produ    | media      | media     
Agradecimiento. Gracias por la atención, todo perfecto                 | baja       | baja      
Caja vacía. Recibí una caja vacía sin el producto dentro               | alta       | baja      
Modificar dirección. Quisiera modificar la dirección de entrega        | media      | media     
Alianza comercial. Nos gustaría evaluar una posible alianza comerci    | baja       | baja      
Web confusa. El servicio fue correcto, pero la web es un poco confu    | media      | baja      
Alta urgent